In [49]:
import duckdb
import pandas as pd

FACT_PATH = "../data/analytics/hmda/fact_loans_2023.parquet"
con = duckdb.connect()


In [50]:
loan_counts = con.execute(f"""
    SELECT loan_purpose, COUNT(*) AS n
    FROM read_parquet('{FACT_PATH}')
    GROUP BY 1
    ORDER BY CAST(loan_purpose AS INTEGER);
""").fetchdf()

loan_counts


,loan_purpose,n
0,1,4791878
1,2,1086065
2,4,1115843
3,5,504
4,31,620899
5,32,1181695


In [51]:
fact_filtered["loan_purpose"].value_counts().sort_index()


loan_purpose
1     4791878
2     1086065
31     620899
32    1181695
4     1115843
Name: count, dtype: int64

In [52]:
agg_purpose = con.execute(f"""
    SELECT
        state_code,
        income_bucket,
        loan_purpose,
        applicant_gender,
        COUNT(*) AS applications,
        SUM(approved)::BIGINT AS approvals,
        AVG(approved)::DOUBLE AS approval_rate,
        quantile_cont(loan_amount, 0.5) AS median_loan_amount
    FROM read_parquet('{FACT_PATH}')
    WHERE applicant_gender IN ('Male','Female')
      AND TRIM(CAST(loan_purpose AS VARCHAR)) != '5'
    GROUP BY 1,2,3,4
""").fetchdf()

agg_purpose.head(10)


,state_code,income_bucket,loan_purpose,applicant_gender,applications,approvals,approval_rate,median_loan_amount
0,CT,Middle,1,Male,11808,8579,0.726541,275000.0
1,TX,Low,32,Female,11683,4847,0.414876,125000.0
2,WA,Middle,1,Female,16383,11975,0.730941,365000.0
3,WA,Middle,1,Male,25964,18719,0.720960,395000.0
4,WA,Low,32,Male,3794,1475,0.388772,205000.0
5,TX,Low,32,Male,13089,5125,0.391550,125000.0
6,IA,Middle,1,Male,11893,9300,0.781973,215000.0
7,IA,Middle,1,Female,6573,5184,0.788681,195000.0
8,MD,Low,1,Female,7411,4456,0.601268,195000.0
9,NJ,Middle,1,Female,17222,11718,0.680409,315000.0


In [56]:
OUT_PATH = "../data/analytics/hmda/agg_gender_income_state_purpose.parquet"
agg_purpose.to_parquet(OUT_PATH, index=False)
print("✅ Saved:", OUT_PATH)


✅ Saved: ../data/analytics/hmda/agg_gender_income_state_purpose.parquet


In [57]:
agg_purpose["loan_purpose"].value_counts().sort_index()


loan_purpose
1     324
2     313
31    317
32    319
4     315
Name: count, dtype: int64